# Probablistic Optimization
This notebook approaches similar problems as the previous notebook, using probability to estimate the profit for each given price. This is quick, computed optimization & hopes to lead to the closed form solution

## The Current Challenge
The challenge I'm currently facing is a difficulty conceptualizing thresholds in a closed form, hence the need for a computerized solution atm.

In [105]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import scipy
from tqdm import tqdm
import plotly.graph_objects as go
import math
import sys
from scipy.stats import norm

In [81]:
# Assumptions: costs is an ascending array, scaling_param is between 0 and 1
def calculate_intervals(prices, costs, tiers, scaling_param):
    
    thresholds = []
    thresholds.append(sys.float_info.min)
    
    for i in range(tiers):
            if i == 0:
                thresholds.append(prices[0]/costs[0])
                                  
            else:
                intersection = (prices[i]-prices[i-1])/(costs[0] * ((costs[i]/costs[0])**(scaling_param)-(costs[i-1]/costs[0])**(scaling_param)))
                j = 0
                while intersection < thresholds[i - j]:
                    j += 1
                    if i == j:
                        intersection = prices[i]/(costs[0]*(costs[i]/costs[0])**(scaling_param))
                        break;
                    else:
                        intersection = (prices[i]-prices[i-j-1])/(costs[0] * ((costs[i]/costs[0])**(scaling_param)-(costs[i-j-1]/costs[0])**(scaling_param)))
                    
                thresholds.append(intersection)
                
                
    for i in range(tiers):
        if (thresholds[tiers - i] < thresholds[tiers-i-1]):
            thresholds[tiers-i-1] = thresholds[tiers - i]
            
    intervals = []
    for i in range(tiers):
        intervals.append((thresholds[i],thresholds[i+1]))
    
    intervals.append((thresholds[len(thresholds)-1],sys.float_info.max))
    return intervals

In [110]:
# Assumptions: costs is an ascending array, scaling_param is between 0 and 1
def tier_probabilities(prices, costs, tiers, intervals, mu, sigma, pdf_type = 'uniform'):
    probabilities = []
    if (pdf_type == 'uniform'):
        start = mu - sigma
        end = mu + sigma
        point_prob = 1/(end - start)
        for i in range(tiers+1):
            probabilities.append((min(intervals[i][1],end)-max(intervals[i][0],start))*point_prob)
    elif (pdf_type == 'gaussian'):
        for i in range(tiers+1):
            probabilities.append(norm.cdf(intervals[i][1],mu,sigma) - norm.cdf(intervals[i][0],mu,sigma))
    else:
        raise ValueError('pdf_type must be uniform or gaussian')
    
    return probabilities
        

In [107]:
# calculate expected profit @ 1 customer based on probability distribution
def profit(prices, costs, tiers, scaling_param, mu, sigma, pdf_type = 'uniform'):
    intervals = calculate_intervals(prices,costs,tiers,scaling_param)
    probabilities = tier_probabilities(prices, costs, tiers, intervals, mu, sigma, pdf_type)
    
    profits = [pr * (p - c) for (p, c, pr) in zip(prices, costs, probabilities[1:])]
    
    return sum(profits)

In [108]:
# negative of profit because scipy likes to minimize
def objective(prices, costs, tiers, scaling_param, mu, sigma, pdf_type = 'uniform'):
    return -profit(prices, costs, tiers, scaling_param, mu, sigma, pdf_type)

In [109]:
C = [1,3,5]
B = len(C);
scaling_lambda = 4/5
mu = 2
sigma = 0.5

bounds = [(1,5),(3,15),(5,20)]

result = scipy.optimize.dual_annealing(objective, bounds, args=(C, B, scaling_lambda, mu, sigma, 'gaussian'))
print(result)

/opt/conda/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1953: RuntimeWarning: overflow encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)


     fun: -1.1688055820319019
 message: ['Maximum number of iteration reached']
    nfev: 6089
    nhev: 0
     nit: 1000
    njev: 22
  status: 0
 success: True
       x: array([1.83415542, 4.68410307, 7.29501182])


0.6676662535792679
